In [ ]:
import pandas as pd
import numpy as np



data = pd.read_csv('../input/higgs-boson/training.zip')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data['Label'].replace(('s','b'),(1,0), inplace=True)

In [ ]:
X = data.iloc[:,1:31]
y = data.iloc[:,-1]

The weights should first be scaled to match the test data since the test data provides the expected number of signal and background events generated by the test set. The test data has 550,000 rows, more than twice the 250,000 rows (len(y)) provided by the training data


In [ ]:
data['test_Weight'] = data['Weight'] * 550000 / len(y)

In [ ]:
s = np.sum(data[data['Label']==1] ['test_Weight'])
b = np.sum(data[data['Label']==0] ['test_Weight'])

In [ ]:
b/s

We scale the weights to match the size of the test data, then divide the sum of the background weights by the sum of the signal weights to
establish the scale_pos_weight=b/s hyperparameter

In [ ]:
import xgboost as xgb

xgb_clf = xgb.DMatrix(X, y, missing=-999.0, weight=data['test_Weight'])

In [ ]:
param = {}

param['objective'] = 'binary:logitraw'
param['scale_pos_weight'] = b/s
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'

In [ ]:
param_list = list(param.items())+[('eval_metric','ams@0.15')]

In [ ]:
watch_list = [ (xgb_clf, 'train')]

In [ ]:
watch_list

In [ ]:
num_round = 120

In [ ]:
print('loading data end, start to boost trees')
bst = xgb.train(param_list, xgb_clf, num_round, watch_list)
bst.save_model('higgs.model')
print('finish training')

The model performs with 94.58 percent auc, and an AMS of 5.9